In [24]:
import os.path
import random
import time
import re
import importlib
import torch
import matplotlib.pyplot as plt
import numpy as np
from unityagents import UnityEnvironment
from collections import deque
import torchvision.transforms as T
from environment import Environment
from agent import DQN_Agent
class Args:
    def __init__(self):
        self.train = True
        self.nographics = False
        self.num_eps = 10
        self.rollout = 5
        self.batchsize = 64
        self.pretrain = 1000
        self.num_episodes = 1
        self.max_time = 50
        self.alr = 1e-4
        self.clr = 1e-4
        self.batch = 128
        self.buffer = 100000
        self.C = 4000        
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")     
        self.pixels = False
        self.eval = False
        self.framework = "DDQN"
        self.prioritized_experience_replay = False
args = Args()
def check_args():
    for arg in vars(args):
        if arg == "sep": continue
        print("{}: {}".format(arg.upper(), getattr(args, arg)))
        
        
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 

In [2]:
import models
import agent
import buffers
import environment
importlib.reload(models)
importlib.reload(agent)
importlib.reload(buffers)
importlib.reload(environment)
from environment import Environment
from agent import DQN_Agent
from models import QNetwork

In [3]:
env = Environment(args, id=1)

##################################################
#                                                #


INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: BananaBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 37
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


#                                                #
##################################################


In [4]:
print(env.state_size, env.action_size, env.agent_count)

37 4 1


In [5]:
env.state.shape

torch.Size([1, 37])

In [78]:
t = torch.rand(env.state.shape)
t

tensor([[0.9827, 0.4103, 0.0498, 0.3412, 0.6834, 0.7162, 0.1646, 0.1478, 0.3577,
         0.5320, 0.0885, 0.7302, 0.4062, 0.7909, 0.6490, 0.4202, 0.4549, 0.6537,
         0.2402, 0.3704, 0.7288, 0.0537, 0.4005, 0.6041, 0.2421, 0.7367, 0.7477,
         0.4275, 0.3393, 0.7420, 0.5282, 0.1134, 0.7882, 0.9922, 0.9279, 0.2242,
         0.9642]])

In [79]:
env.state

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.8441, 0.0000, 0.0000, 1.0000, 0.0000,
         0.0748, 0.0000, 1.0000, 0.0000, 0.0000, 0.2576, 1.0000, 0.0000, 0.0000,
         0.0000, 0.7418, 0.0000, 1.0000, 0.0000, 0.0000, 0.2585, 0.0000, 0.0000,
         1.0000, 0.0000, 0.0936, 0.0000, 1.0000, 0.0000, 0.0000, 0.3197, 0.0000,
         0.0000]])

In [6]:
q = QNetwork(env.state_size, env.action_size, 0)

In [121]:
states = [torch.rand(env.state.shape) for _ in range(5)]
next_states = [torch.rand(env.state.shape) for _ in range(5)]
states = torch.cat(states)
next_states = torch.cat(next_states)
print(next_states.shape)

torch.Size([5, 37])


In [81]:
states.shape

torch.Size([5, 37])

In [123]:
a = q(env.state)
print(a.max(dim=1)[1].view(1,1))
print(a.argmax(dim=1).unsqueeze(0))

tensor([[2]])
tensor([[2]])


In [115]:
b = q(states).detach().argmax(1).unsqueeze(1)
print(b)
print(b.shape)
b[2][0] = 1
b[1][0] = 0
print(b)

tensor([[2],
        [2],
        [2],
        [2],
        [2]])
torch.Size([5, 1])
tensor([[2],
        [0],
        [1],
        [2],
        [2]])


In [116]:
v = q(next_states).detach()
print(v.shape)
print(v)

torch.Size([5, 4])
tensor([[ 0.0177, -0.0694,  0.1381,  0.0422],
        [ 0.0564, -0.0554,  0.0868,  0.0523],
        [-0.0253, -0.0900,  0.1249,  0.0320],
        [ 0.0205, -0.0707,  0.1340,  0.0234],
        [ 0.0372, -0.0809,  0.1088,  0.0099]])


In [120]:
v2 = v.gather(1, b)
v2 = v2.squeeze(1)
print(v2)
print(v2.shape)

tensor([ 0.1381,  0.0564, -0.0900,  0.1340,  0.1088])
torch.Size([5])


In [124]:
torch.LongTensor(1).random_(4).unsqueeze(0)

tensor([[2]])

In [158]:
n = np.random.randint(0,3, size=(1,1))

array([[1]])

In [ ]:
env.close()